In [1]:
import torch
from transformers import AutoTokenizer, RobertaForQuestionAnswering
import numpy as np

## Load SQuad Data

SQuad dataset fields
- answers: the starting location of the answer token and the answer text.
- context: background information from which the model needs to extract the answer.
- question: the question a model should answer.

In [2]:
import pandas as pd
from datasets import load_dataset

In [3]:
squad = load_dataset("squad_v2")

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

### Squad preprocess

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [5]:
pad_on_right = tokenizer.padding_side == "right"
# 
def preprocess_function(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
                
    return tokenized_examples

In [6]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [7]:
tokenized_squad

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 131823
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 12165
    })
})

In [8]:
from transformers import default_data_collator

data_collator = default_data_collator

2024-08-08 01:07:48.185888: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 01:07:48.186021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 01:07:48.378304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
tokenized_squad

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 131823
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 12165
    })
})

## Load Model

In [10]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

roberta_model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [11]:
from transformers import pipeline
question = "How many languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
question_answerer(question=question, context=context)

{'score': 0.6467295289039612, 'start': 58, 'end': 60, 'answer': '46'}

## Evaluation

Link: https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb

In [12]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
        
    tokenized_examples["input_ids"] = [
        [token_id for token_id in input_ids if token_id is not None]
        for input_ids in tokenized_examples["input_ids"]
    ]

    return tokenized_examples

In [13]:
validation_set = squad["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=squad["validation"].column_names,
)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [14]:
validation_features = validation_set.remove_columns(["example_id", "offset_mapping"])

In [15]:
validation_features.set_format("torch")

In [16]:
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
roberta_model.to(device)
batch_size=16
start_logits = None
end_logits = None


# for i in tqdm(range(0, len(validation_features), batch_size)):
#     batch = validation_features[i:i+batch_size]
#     batch = {k: torch.tensor(v).to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = roberta_model(**batch)
        
#     if start_logits is None:
#         start_logits = outputs.start_logits.cpu().numpy()
#     else:
#         start_logits = np.append(start_logits, outputs.start_logits.cpu().numpy(), axis=0)
        
#     if end_logits is None:
#         end_logits = outputs.end_logits.cpu().numpy()
#     else:
#         end_logits = np.append(end_logits, outputs.end_logits.cpu().numpy(), axis=0)

In [17]:
# save outputs
# np.save("roberta_eval_start_logits.npy", start_logits)
# np.save("roberta_eval_end_logits.npy", end_logits)

In [18]:
# load start_logits
start_logits = np.load("/kaggle/input/distillbert-output/roberta_eval_start_logits.npy")
end_logits = np.load("/kaggle/input/distillbert-output/roberta_eval_end_logits.npy")

In [19]:
import numpy as np
from datasets import load_metric
import collections
from tqdm.auto import tqdm

n_best = 15
max_answer_length = 30
is_squadv2 = True

def compute_answer(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    na_probs = {}
    
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []
        
        min_null_score = None # Only used if squad_v2 is True.

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]
            
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logit[cls_index] + end_logit[cls_index]
            
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)
        
        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
        else:
            best_answer = {"text": "", "logit_score": 0.0}
        
        if is_squadv2:
            best_answer_text = best_answer["text"] if best_answer["logit_score"] > min_null_score else ""
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer_text}
            )
        else:
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        
    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return predicted_answers, theoretical_answers

In [20]:
predicted_answers, theoretical_answers = compute_answer(start_logits, end_logits, validation_set, squad["validation"])

  0%|          | 0/11873 [00:00<?, ?it/s]

In [21]:
theoretical_answers[0:10]

[{'id': '56ddde6b9a695914005b9628',
  'answers': {'text': ['France', 'France', 'France', 'France'],
   'answer_start': [159, 159, 159, 159]}},
 {'id': '56ddde6b9a695914005b9629',
  'answers': {'text': ['10th and 11th centuries',
    'in the 10th and 11th centuries',
    '10th and 11th centuries',
    '10th and 11th centuries'],
   'answer_start': [94, 87, 94, 94]}},
 {'id': '56ddde6b9a695914005b962a',
  'answers': {'text': ['Denmark, Iceland and Norway',
    'Denmark, Iceland and Norway',
    'Denmark, Iceland and Norway',
    'Denmark, Iceland and Norway'],
   'answer_start': [256, 256, 256, 256]}},
 {'id': '56ddde6b9a695914005b962b',
  'answers': {'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],
   'answer_start': [308, 308, 308, 308]}},
 {'id': '56ddde6b9a695914005b962c',
  'answers': {'text': ['10th century',
    'the first half of the 10th century',
    '10th',
    '10th'],
   'answer_start': [671, 649, 671, 671]}},
 {'id': '5ad39d53604f3c001a3fe8d1',
  'answers': {'text': [], 'answe

In [22]:
squad["validation"][7]

{'id': '5ad39d53604f3c001a3fe8d3',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'Who did King Charles III swear fealty to?',
 'answers': {'text': [], 'answer_start': []}}

In [23]:
predicted_answers[0:10]

[{'id': '56ddde6b9a695914005b9628', 'prediction_text': 'France'},
 {'id': '56ddde6b9a695914005b9629',
  'prediction_text': '10th and 11th centuries'},
 {'id': '56ddde6b9a695914005b962a',
  'prediction_text': 'Denmark, Iceland and Norway'},
 {'id': '56ddde6b9a695914005b962b', 'prediction_text': ''},
 {'id': '56ddde6b9a695914005b962c', 'prediction_text': '10th'},
 {'id': '5ad39d53604f3c001a3fe8d1', 'prediction_text': ''},
 {'id': '5ad39d53604f3c001a3fe8d2', 'prediction_text': ''},
 {'id': '5ad39d53604f3c001a3fe8d3', 'prediction_text': ''},
 {'id': '5ad39d53604f3c001a3fe8d4', 'prediction_text': ''},
 {'id': '56dddf4066d3e219004dad5f',
  'prediction_text': 'William the Conqueror'}]

In [24]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [25]:
import evaluate

metric = evaluate.load("squad_v2")

In [26]:
print(predicted_answers[0])
print(theoretical_answers[0])

{'id': '56ddde6b9a695914005b9628', 'prediction_text': 'France'}
{'id': '56ddde6b9a695914005b9628', 'answers': {'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}}


In [27]:
if is_squadv2:
    formatted_predictions = [{"id": ans["id"], "prediction_text": ans["prediction_text"], "no_answer_probability": 0.0} for ans in predicted_answers]

In [28]:
metric.compute(predictions=formatted_predictions, references=theoretical_answers)

{'exact': 79.92082877116145,
 'f1': 82.91749890730917,
 'total': 11873,
 'HasAns_exact': 77.93522267206478,
 'HasAns_f1': 83.93715663402219,
 'HasAns_total': 5928,
 'NoAns_exact': 81.90075693860386,
 'NoAns_f1': 81.90075693860386,
 'NoAns_total': 5945,
 'best_exact': 79.92082877116145,
 'best_exact_thresh': 0.0,
 'best_f1': 82.91749890730902,
 'best_f1_thresh': 0.0}